In [0]:
import pandas as pd
import string
import nltk
import numpy as np
import sklearn
import seaborn as sns
import re
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import minmax_scale
from scipy import sparse
from scipy.sparse import hstack
from sklearn.metrics import classification_report,accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/train_newb.csv")
valid_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/val_newb.csv")
test_data=pd.read_csv("/content/drive/My Drive/ML project'20/YelpZip/test_newb.csv")

In [0]:
Days={}
i=1
for val in train_data['day']:
  if val not in Days:
    Days[val]= i
    i+=1
train_data['day']=train_data['day'].apply(lambda x: Days[x])
test_data['day']=test_data['day'].apply(lambda x: Days[x])
valid_data['day']=valid_data['day'].apply(lambda x: Days[x])

In [0]:
train_data['label']=train_data['label'].apply(lambda x: 0 if x==-1 else 1)
test_data['label']=test_data['label'].apply(lambda x: 0 if x==-1 else 1)
valid_data['label']=valid_data['label'].apply(lambda x: 0 if x==-1 else 1)
Days={}
i=1
for val in train_data['day']:
  if val not in Days:
    Days[val]= i
    i+=1
train_data['day']=train_data['day'].apply(lambda x: Days[x])
test_data['day']=test_data['day'].apply(lambda x: Days[x])
valid_data['day']=valid_data['day'].apply(lambda x: Days[x])

In [0]:
train_data.head()

,user_id,product_id,rating,label,date,review_text,day,year,word_count,punctuation_count,char_count,title_count,user_id_no_of_review,user_id_ave_rating,user_id_ave_no_words,user_id_max_review_a_day,product_id_no_of_review,product_id_ave_rating,product_id_ave_no_of_words,product_id_max_review_a_day
0,192117,3237,4.0,0,2012-12-30,"delish! pizza was huge, flavorful and filling!...",1,2012,52,8,296,7,1,4.0,52.0,1,114,4.017544,74.684211,14
1,83124,2122,4.0,1,2012-01-21,lava's hummus is better than zahav's. there. i...,2,2012,225,34,1233,21,1,4.0,225.0,1,15,3.666667,140.266667,3
2,137167,3222,2.0,1,2012-08-10,my friend and i decided to come check this pla...,3,2012,247,21,1315,11,1,2.0,247.0,1,32,3.500000,82.656250,4
3,51095,523,3.0,1,2012-04-23,oy vey. i really really wanted to write a good...,4,2012,210,52,1184,22,1,3.0,210.0,1,18,3.111111,107.611111,6
4,194284,2605,4.0,0,2012-03-22,i heard about the wait and it was worth it. th...,5,2012,21,4,95,2,2,3.5,25.5,2,117,3.837607,124.427350,15


In [0]:
X_train=train_data.drop(['user_id','product_id','year','date','label','rating','review_text'],axis=1)
y_train=train_data.label.values
X_test=test_data.drop(['user_id','product_id','year','date','label','rating','review_text'],axis=1)
y_test=test_data.label.values
X_valid=valid_data.drop(['user_id','product_id','year','date','label','rating','review_text'],axis=1)
y_valid=valid_data.label.values

In [0]:
pd.set_option('mode.chained_assignment', None)

In [0]:
feat=1000
count_vect_1 = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1))
count_vect_1.fit(train_data.review_text.values)
pos=count_vect_1.transform(train_data.review_text[train_data.label==1])
neg=count_vect_1.transform(train_data.review_text[train_data.label==0])
val=(neg.sum(axis=0)-pos.sum(axis=0))
ind_2=np.squeeze(np.asarray(val)).argsort()[-feat:][::-1]
va_1l=(pos.sum(axis=0)-neg.sum(axis=0))
ind_1=np.squeeze(np.asarray(va_1l)).argsort()[-feat:][::-1]
ind=np.concatenate((ind_1,ind_2))
voc=np.array(count_vect_1.get_feature_names())
vocab=dict(np.ndenumerate(voc[ind]))
xtrain_count_1=count_vect_1.transform(train_data.review_text)
xtrain_count_1=xtrain_count_1[:,ind]
xtest_count_1=count_vect_1.transform(test_data.review_text)
xtest_count_1=xtest_count_1[:,ind]
xvalid_count_1 =count_vect_1.transform(valid_data.review_text)
xvalid_count_1=xvalid_count_1[:,ind]

In [0]:
xtrain_count_1

<25000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 1362751 stored elements in Compressed Sparse Row format>

In [0]:
#Convert text features into 
import torch
import numpy as np
from scipy.sparse import coo_matrix

# Convert into Tensor sparse matrix
values = xtrain_count_1.data
indices = np.vstack((xtrain_count_1.tocoo().row, xtrain_count_1.tocoo().col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = xtrain_count_1.shape

train_features_tensor=torch.sparse.FloatTensor(i, v, torch.Size(shape))

#Valid
values = xvalid_count_1.data
indices = np.vstack((xvalid_count_1.tocoo().row, xvalid_count_1.tocoo().col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = xtrain_count_1.shape

valid_features_tensor=torch.sparse.FloatTensor(i, v, torch.Size(shape))


#test
values = xtest_count_1.data
indices = np.vstack((xtest_count_1.tocoo().row, xtest_count_1.tocoo().col))

i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = xtrain_count_1.shape

test_features_tensor=torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [0]:
from torch.utils.data import Dataset,DataLoader
class MycustomDataset(Dataset):
  def __init__(self,text_features,meta_features,y):
    super().__init__()
    
    self.metafeatures=meta_features
    self.textfeatures=text_features
    self.y=y
  def __len__(self):
    return len(self.y)
  def __getitem__(self,idx):
    return self.textfeatures[idx],self.metafeatures[idx],self.y[idx]

In [0]:
train_features_tensor[0]

tensor(indices=tensor([[ 401,  298, 1057,    2,   42,    0,   82, 1003, 1002,
                         190, 1007, 1006,   95,  787,    4, 1184,  254,   54,
                           5,    3,  201, 1001, 1133,  142,  185, 1009,  488,
                          36,    6, 1581,   46,    8, 1040,    1]]),
       values=tensor([0.1710, 0.1401, 0.0889, 0.1770, 0.0510, 0.0839, 0.1306,
                      0.1172, 0.0846, 0.1533, 0.0700, 0.1322, 0.0894, 0.1747,
                      0.1121, 0.2214, 0.1210, 0.0876, 0.0557, 0.0979, 0.1525,
                      0.2298, 0.1334, 0.1700, 0.1917, 0.1556, 0.2123, 0.1136,
                      0.0626, 0.1667, 0.0863, 0.1739, 0.1208, 0.0477]),
       size=(2000,), nnz=34, layout=torch.sparse_coo)

In [0]:
train_dataset=MycustomDataset(train_features_tensor,torch.tensor(X_train.values),torch.tensor(y_train))
train_iterator=DataLoader(train_dataset,batch_size=256,shuffle=True)
valid_dataset=MycustomDataset(valid_features_tensor,torch.tensor(X_valid.values),torch.tensor(y_valid))
valid_iterator=DataLoader(valid_dataset,batch_size=256,shuffle=True)
test_dataset=MycustomDataset(test_features_tensor,torch.tensor(X_test.values),torch.tensor(y_test))
test_iterator=DataLoader(test_dataset,batch_size=256,shuffle=True)

In [0]:
len(train_iterator)

98

In [0]:
for i in valid_iterator:
  print(i[0])
  print(i[0].shape)
  print(i[1].shape)
  print(i[2].shape)
  break

tensor(indices=tensor([[  0,   0,   0,  ..., 255, 255, 255],
                       [ 92,  44,   7,  ..., 172,   1,  86]]),
       values=tensor([0.1067, 0.1741, 0.0624,  ..., 0.0299, 0.0584, 0.0407]),
       size=(256, 2000), nnz=13345, layout=torch.sparse_coo)
torch.Size([256, 2000])
torch.Size([256, 13])
torch.Size([256])


### **Neural Network Model**

In [0]:
import torch 
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [0]:
class NeuralNetwork(nn.Module):
  def __init__(self,batch_size,hidden_size1,hidden_size2,hidden_size3,hidden_size4,outputsize,metadata_length,text_length,keep_probab):
    super().__init__()
    self.input_size=batch_size
    self.feature_size_text= text_length #1000
    self.feature_size_metadata=metadata_length
    self.hidden_size1=hidden_size1
    self.hidden_size2=hidden_size2
    self.hidden_size3=hidden_size3
    self.hidden_size4=hidden_size4
    self.output_size=output_size
    self.layer1=nn.Linear(text_length,self.hidden_size1)
    self.layer2=nn.Linear(self.hidden_size1,self.hidden_size1)
    self.layer3=nn.Linear(self.hidden_size1,self.hidden_size2)
    self.layer4=nn.Linear(metadata_length,self.hidden_size3)
    self.layer5=nn.Linear(self.hidden_size3,self.hidden_size3)
    self.layer6=nn.Linear(self.hidden_size3,self.hidden_size4)
    self.out_layer=nn.Linear(self.hidden_size4+self.hidden_size2,self.output_size)
    self.relu=nn.ReLU()
    self.dropout=nn.Dropout(keep_probab)
    self.softmax=nn.Softmax()

  def forward(self,input_text,input_features):
    # 3 hidden layer neural network for review text.
    z=self.layer1(input_text)
    z=self.dropout(z)
    z=self.relu(z)
    z1=self.layer2(z)
    z1=self.dropout(z1)
    z1=self.relu(z1)
    z2=self.layer3(z1)
    z2=self.relu(z2)
    out1=self.dropout(z2)

    #3 hidden layer neural network for metadata features
    z3=self.layer4(input_features)
    z3=self.dropout(z3)
    z3=self.relu(z3)
    z4=self.layer5(z3)
    z4=self.dropout(z3)
    z4=self.relu(z4)
    z5=self.layer6(z4)
    z5=self.relu(z5)
    out2=self.dropout(z5)
    
    #Output Layer
    self.out=torch.cat((out1,out2),dim=1)
    # self.out=self.dropout(self.out)
    self.out=self.out_layer(self.out)
    output=self.softmax(self.out)
    return output

In [0]:
batch_size=256
hidden_size1=1000
hidden_size2=400
hidden_size3=10
hidden_size4=5
text_length=2000
metadata_length=13
output_size=2
keep_probab=0.2
NN=NeuralNetwork(batch_size,hidden_size1,hidden_size2,hidden_size3,hidden_size4,output_size,metadata_length,text_length,keep_probab)

In [0]:
#architecture
print(NN)

#No. of trianable parameters
def count_parameters(NN):
    return sum(p.numel() for p in NN.parameters() if p.requires_grad)

print(f'The model has {count_parameters(NN):,} trainable parameters')

NeuralNetwork(
  (layer1): Linear(in_features=2000, out_features=1000, bias=True)
  (layer2): Linear(in_features=1000, out_features=1000, bias=True)
  (layer3): Linear(in_features=1000, out_features=400, bias=True)
  (layer4): Linear(in_features=13, out_features=10, bias=True)
  (layer5): Linear(in_features=10, out_features=10, bias=True)
  (layer6): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=405, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=None)
)
The model has 3,403,517 trainable parameters


In [0]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#define optimizer and loss
optimizer = optim.Adam(NN.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
# model = model.to(device)
criterion = criterion.to(device)

In [0]:
# train function for Bi-LSTM

def train(NN, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    predicted=[]
    truth=[]
    #set the model in training phase
    NN.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        # input_text = batch[-14:]
        # metadata=batch[0:10000]
        #convert to 1D tensor
        predictions = NN(batch[0], batch[1].float())  
        
        #compute the loss
        loss = criterion(predictions[:,1], batch[2].float())        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions[:,1], batch[2])   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        predicted += list(predictions[:,1].detach().cpu().numpy().round())
        truth += list(batch[2].detach().cpu().numpy())

    f1 = f1_score(truth, predicted)
    return epoch_loss / len(iterator), epoch_acc / len(iterator),f1

In [0]:
from sklearn.metrics import f1_score
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0
    predicted=[]
    truth=[]
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            # text, text_lengths = batch.headline
            
            #convert to 1d tensor
            predictions = model(batch[0],batch[1].float())
            #compute loss and accuracy
            loss = criterion(predictions[:,1], batch[2].float())
            acc = binary_accuracy(predictions[:,1], batch[2])
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

            predicted += list(predictions[:,1].detach().cpu().numpy().round())
            truth += list(batch[2].detach().cpu().numpy())
            
    f1 = f1_score(truth, predicted)
    report=classification_report(truth,predicted)
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1,report

In [0]:
#p=0.2
N_EPOCHS = 3
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc,f1_train = train(NN, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss,valid_acc,f1_valid,classification__report=evaluate(NN,valid_iterator,criterion)
    test_loss, test_acc,f1_test,classification__report = evaluate(NN, test_iterator, criterion)

    #save the best model
    if test_loss < best_valid_loss:
        best_valid_loss = test_loss
        torch.save(NN.state_dict(), 'saved_weights_cnn.pt')
    print("For Epoch {}".format(epoch+1))
    print(f'\tTrain Loss: {train_loss:.3f}   | Train Acc: {train_acc*100:.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f}   | Valid Acc: {valid_acc*100:.2f}%')
    print(f'\tTest  Loss: {test_loss:.3f}   |  Test  Acc: {test_acc*100:.2f}%')
    print(f'\tF1 train  : {f1_train:.4f}    | F1 test  : {f1_test:.4f} | F1 Valid  : {f1_valid:.4f}')
    print('\033[1m'+"Classification Report"+'\033[0m')
    print(classification__report)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


For Epoch 1
	Train Loss: 0.647   | Train Acc: 62.52%
	Valid Loss: 0.626   | Valid Acc: 64.63%
	Test  Loss: 0.612   |  Test  Acc: 65.49%
	F1 train  : 0.6027    | F1 test  : 0.6300 | F1 Valid  : 0.6120
Classification Report
              precision    recall  f1-score   support

           0       0.64      0.74      0.68      5000
           1       0.69      0.58      0.63      5000

    accuracy                           0.66     10000
   macro avg       0.66      0.66      0.66     10000
weighted avg       0.66      0.66      0.66     10000

For Epoch 2
	Train Loss: 0.553   | Train Acc: 72.18%
	Valid Loss: 0.632   | Valid Acc: 65.21%
	Test  Loss: 0.616   |  Test  Acc: 66.23%
	F1 train  : 0.7163    | F1 test  : 0.6588 | F1 Valid  : 0.6456
Classification Report
              precision    recall  f1-score   support

           0       0.66      0.68      0.67      5000
           1       0.67      0.65      0.66      5000

    accuracy                           0.66     10000
   macro av